```
[2] 추출 대상 대장 : 건축, 주택 허가대장
  - 가설건축물, 기타 용도 제외
[3] 추출 기준
  1. 허가일/착공일 : 2023.03.01 ~ 2023.03.31.
  2. 동수 : 층 대표 용도 방식으로 집계
     2.1 건물내 층별 용도의 유형별 연면적 합 추출
     2.2 가장 넓은 면적의 층용도를 동의 층 대표용도로 추출
     2.3 동의 층 대표용도 기준으로 동수 집계
  3. 연면적 : 층 용도별 면적의 합
  4. 용도 : 층별 용도
  5. 구조 : 동별 구조
     5.1 기타 : 기타구조 + 동별 구조(항목) 값 오기 또는 누락
  6. 건축주(사업주체) : 대표 건축주의 건축주 구분
     6.1 기타 : 대표 건축주의 건축주 구분(항목) 값 오기 또는 누락
"[4] 건축통계는 허가취소 및 허가사항 정정 등 업무 특성으로 
     자료기준일에 따라 수치가 다르니, 이용시 참고하시기 바랍니다."
```

- 층 용도별 면적의 합
- 건축인허가
  - 면적은 층별 층면적 사용
  - 행위는 층별 건축구분 사용
  - 용도는 층별 주용도 사용
  - 구조는 동별 사용
  - 시군구는 층별 사용(어차피 기본에서 온 것)
  - 허가일은 기본 건축허가일 사용
  - 착공일은 기본 실제착공일 사용
  - 준공일은 기본 사용승인일 사용
- 주택인허가
  - 면적은 층별 층면적 사용
  - 주택인허가 행위는 모두 신축(출처: 건축허가 통계 쿼리)
  - 용도는 층별 용도 사용
  - 구조는 동별 구조 사용
  - 시군구는 층별 사용(어차피 기본에서 온 것)
  - 허가일은 승인일 사용
  - 착공일은 착공일 사용? 없는데??
  - 준공일은 기본 사용검사일 사용

In [1]:
import polars as pl
from pathlib import Path
from pprint import pprint

df_master_building = pl.scan_parquet("data/건축인허가_기본개요_2022년_12월/*")
df_dong_building = pl.scan_parquet("data/건축인허가_동별개요_2022년_12월/*")
df_floor_building = pl.scan_parquet("data/건축인허가_층별개요_2022년_12월/*")


In [2]:
print(df_master_building.columns)
print(df_dong_building.columns)
print(df_floor_building.columns)


['관리_허가대장_PK', '대지_위치', '건물_명', '시군구_코드', '법정동_코드', '대지_구분_코드', '번', '지', '특수지_명', '블록', '로트', '지목_코드_명', '지역_코드_명', '지구_코드_명', '구역_코드_명', '지목_코드', '지역_코드', '지구_코드', '구역_코드', '건축_구분_코드', '건축_구분_코드_명', '대지_면적(㎡)', '건축_면적(㎡)', '건폐_율(%)', '연면적(㎡)', '용적_률_산정_연면적(㎡)', '용적_률(%)', '주_건축물_수', '부속_건축물_동_수', '주_용도_코드', '주_용도_코드_명', '세대_수(세대)', '호_수(호)', '가구_수(가구)', '총_주차_수', '착공_예정_일', '착공_연기_일', '실제_착공_일', '건축_허가_일', '사용승인_일', '생성_일자', '__null_dask_index__']
['관리_동별_개요_PK', '관리_허가대장_PK', '대지_위치', '건물_명', '시군구_코드', '법정동_코드', '대지_구분_코드', '번', '지', '특수지_명', '블록', '로트', '주_부속_구분_코드', '주_부속_구분_코드_명', '주_용도_코드', '주_용도_코드_명', '호_수(호)', '가구_수(가구)', '세대_수(세대)', '구조_코드', '구조_코드_명', '지붕_코드', '지붕_코드_명', '건축_면적(㎡)', '연면적(㎡)', '용적_률_산정_연면적(㎡)', '생성_일자', '승용_승강기_수', '비상용_승강기_수', '__null_dask_index__']
['관리_층별_개요_PK', '관리_동별_개요_PK', '관리_허가대장_PK', '대지_위치', '건물_명', '시군구_코드', '법정동_코드', '대지_구분_코드', '번', '지', '특수지_명', '블록', '로트', '구조_코드', '구조_코드_명', '주_용도_코드', '주_용도_코드_명', '층_번호', '층_면적(㎡)', '층_구분_코드', '층_구분_코드_명',

In [3]:
cols_master_building = [
    "관리_허가대장_PK",
    # "시군구_코드",
    # "건축_구분_코드",
    # "건축_구분_코드_명",
    "실제_착공_일",
    "건축_허가_일",
    "사용승인_일",
    # "생성_일자",
]
cols_dong_building = [
    "관리_동별_개요_PK",
    "관리_허가대장_PK",
    # "시군구_코드",
    "구조_코드",
    "구조_코드_명",
    # "생성_일자",
]
cols_floor_building = [
    "관리_층별_개요_PK",
    "관리_동별_개요_PK",
    "관리_허가대장_PK",
    "시군구_코드",
    "주_용도_코드",
    "주_용도_코드_명",
    "층_번호",
    "층_면적(㎡)",
    "층_구분_코드",
    "층_구분_코드_명",
    "건축_구분_코드",
    "건축_구분_코드_명",
    # "생성_일자",   # null이 없긴 한데 믿을 게 못 됨
]


In [4]:
df_master_building.select(cols_master_building).collect().describe()


describe,관리_허가대장_PK,실제_착공_일,건축_허가_일,사용승인_일
str,str,str,str,str
"""count""","""5770225""","""5770225""","""5770225""","""5770225"""
"""null_count""","""0""","""2661896""","""41033""","""2726077"""
"""mean""",null,null,null,null
"""std""",null,null,null,null
"""min""","""-1000000000000…","""00""","""- -""","""0"""
"""max""","""50130-9999""","""997011""","""997011""","""99990619"""
"""median""",null,null,null,null
"""25%""",null,null,null,null
"""75%""",null,null,null,null


In [5]:
df_dong_building.select(cols_dong_building).collect().describe()


describe,관리_동별_개요_PK,관리_허가대장_PK,구조_코드,구조_코드_명
str,str,str,str,str
"""count""","""7379512""","""7379512""","""7379512""","""7379512"""
"""null_count""","""0""","""0""","""37554""","""37557"""
"""mean""",null,null,null,null
"""std""",null,null,null,null
"""min""","""-1000000000000…","""-1000000000000…","""08""","""강파이프구조"""
"""max""","""50130-9999""","""50130-9999""","""99""","""흙벽돌조"""
"""median""",null,null,null,null
"""25%""",null,null,null,null
"""75%""",null,null,null,null


In [6]:
df_floor_building.select(cols_floor_building).collect().describe()


describe,관리_층별_개요_PK,관리_동별_개요_PK,관리_허가대장_PK,시군구_코드,주_용도_코드,주_용도_코드_명,층_번호,층_면적(㎡),층_구분_코드,층_구분_코드_명,건축_구분_코드,건축_구분_코드_명
str,str,str,str,str,str,str,f64,f64,str,str,str,str
"""count""","""13332779""","""13332779""","""13332779""","""13332779""","""13332779""","""13332779""",1.3332779e7,1.3332779e7,"""13332779""","""13332779""","""13332779""","""13332779"""
"""null_count""","""0""","""0""","""0""","""4154""","""181669""","""183494""",723.0,0.0,"""402""","""402""","""312935""","""314899"""
"""mean""",null,null,null,null,null,null,2.113716,225.345542,null,null,null,null
"""std""",null,null,null,null,null,null,5.984447,52064.581632,null,null,null,null
"""min""","""-1000000000000…","""-1000000000000…","""-1000000000000…","""03000""","""01000""","""가상현실체험제공업소""",-2.0,-69575.0,"""10""","""복수층(상층)""","""0100""","""가설건축물축조허가"""
"""max""","""50130-9999""","""50130-9999""","""50130-9999""","""50130""","""n3""","""휴양콘도미니엄""",7011.0,1.87102521e8,"""30""","""지하""","""3000""","""허가/신고사항변경"""
"""median""",null,null,null,null,null,null,1.0,92.02,null,null,null,null
"""25%""",null,null,null,null,null,null,1.0,27.16,null,null,null,null
"""75%""",null,null,null,null,null,null,2.0,170.9,null,null,null,null


In [7]:
ktng = (
    (pl.col("시군구_코드") == "36110")
    & (pl.col("법정동_코드") == "11000")
    & (pl.col("대지_구분_코드") == "0")
    & (pl.col("번") == "0524")
    & (pl.col("지") == "0000")
)


In [8]:
df_master_building.filter(ktng).select(cols_master_building).collect()


관리_허가대장_PK,실제_착공_일,건축_허가_일,사용승인_일
str,str,str,str
"""36110-50001049…","""20201013""","""20200917""","""20201203"""
"""36110-50001091…",null,"""20201214""","""20210113"""
"""36110-50000461…","""20161220""","""20160712""","""20200528"""


In [9]:
df_dong_building.filter(ktng).select(cols_dong_building).collect()


관리_동별_개요_PK,관리_허가대장_PK,구조_코드,구조_코드_명
str,str,str,str
"""36110-50001942…","""36110-50001049…","""21""","""철근콘크리트구조"""
"""36110-50001972…","""36110-50001091…","""21""","""철근콘크리트구조"""
"""36110-50001799…","""36110-50000461…","""21""","""철근콘크리트구조"""


In [10]:
df_floor_building.filter(ktng & ((pl.col("층_번호").is_in([3, 6])))).select(
    cols_floor_building
).collect().sort(
    ["층_구분_코드", "층_번호"]
)  # 대수선, 용도변경 층만 선택


관리_층별_개요_PK,관리_동별_개요_PK,관리_허가대장_PK,시군구_코드,주_용도_코드,주_용도_코드_명,층_번호,층_면적(㎡),층_구분_코드,층_구분_코드_명,건축_구분_코드,건축_구분_코드_명
str,str,str,str,str,str,i64,f64,str,str,str,str
"""36110-50001554…","""36110-50001942…","""36110-50001049…","""36110""","""14204""","""사무소""",3,821.31,"""10""","""지하""","""0600""","""대수선"""
"""36110-50001554…","""36110-50001942…","""36110-50001049…","""36110""","""14204""","""사무소""",3,3995.86,"""10""","""지하""","""0600""","""대수선"""
"""36110-50001433…","""36110-50001799…","""36110-50000461…","""36110""","""14204""","""사무소""",3,685.37,"""10""","""지하""","""0100""","""신축"""
"""36110-50001433…","""36110-50001799…","""36110-50000461…","""36110""","""14204""","""사무소""",3,4112.7,"""10""","""지하""","""0100""","""신축"""
"""36110-50001433…","""36110-50001799…","""36110-50000461…","""36110""","""07201""","""상점""",3,1841.04,"""20""","""지상""","""0100""","""신축"""
"""36110-50001433…","""36110-50001799…","""36110-50000461…","""36110""","""14204""","""사무소""",6,517.89,"""20""","""지상""","""0100""","""신축"""
"""36110-50001433…","""36110-50001799…","""36110-50000461…","""36110""","""14204""","""사무소""",6,1457.45,"""20""","""지상""","""0100""","""신축"""
"""36110-50001579…","""36110-50001972…","""36110-50001091…","""36110""","""10002""","""직업훈련소""",6,1457.45,"""20""","""지상""","""0700""","""용도변경"""


In [11]:
new = ["0100"]  # 신축
mod = ["0200", "0300", "0500", "0600"]  # 증축/개축/이전/대수선
change_of_use = ["0700"]  # 용도변경


In [55]:
df_result_building = (
    df_master_building.select(cols_master_building)
    .filter(  # 2022년까지 데이터만
        (pl.col("건축_허가_일").str.lengths() == 8) & (pl.col("건축_허가_일") <= "20221231")
    )
    .join(df_dong_building.select(cols_dong_building), "관리_허가대장_PK", suffix="_dong")
    .join(df_floor_building.select(cols_floor_building), "관리_동별_개요_PK", suffix="_floor")
)


In [56]:
df_result_building.head().collect()


관리_허가대장_PK,실제_착공_일,건축_허가_일,사용승인_일,관리_동별_개요_PK,구조_코드,구조_코드_명,관리_층별_개요_PK,관리_허가대장_PK_floor,시군구_코드,주_용도_코드,주_용도_코드_명,층_번호,층_면적(㎡),층_구분_코드,층_구분_코드_명,건축_구분_코드,건축_구분_코드_명
str,str,str,str,str,str,str,str,str,str,str,str,i64,f64,str,str,str,str
"""46800-938""","""20051125""","""20051121""","""20060324""","""46800-1339""","""33""","""강파이프구조""","""46800-1710""","""46800-938""","""46800""","""21101""","""축사""",1,396.0,"""20""","""지상""","""0100""","""신축"""
"""29110-12000000…","""20120627""","""20111004""","""20121203""","""29110-12000543…","""21""","""철근콘크리트구조""","""29110-12000594…","""29110-12000000…","""29110""","""01002""","""다중주택""",3,135.0,"""20""","""지상""","""0100""","""신축"""
"""29110-12000000…","""20120627""","""20111004""","""20121203""","""29110-12000543…","""21""","""철근콘크리트구조""","""29110-12000594…","""29110-12000000…","""29110""","""01002""","""다중주택""",2,135.0,"""20""","""지상""","""0100""","""신축"""
"""29110-12000000…","""20120627""","""20111004""","""20121203""","""29110-12000543…","""21""","""철근콘크리트구조""","""29110-12000594…","""29110-12000000…","""29110""","""01002""","""다중주택""",1,57.94,"""20""","""지상""","""0100""","""신축"""
"""29110-12000004…","""20111007""","""20111005""","""20120322""","""29110-12000212…","""21""","""철근콘크리트구조""","""29110-12000243…","""29110-12000004…","""29110""","""01003""","""다가구주택""",1,12.36,"""20""","""지상""","""0100""","""신축"""


In [15]:
# df_permit_building = (
#     df_result_building.filter(pl.col("건축_구분_코드").is_in(new + mod))  # 신축과 증개이대만
#     .with_columns((pl.col("건축_허가_일").str.slice(0, 6).alias("연월")))
#     .groupby("연월")
#     .agg(pl.col("층_면적(㎡)").sum().alias("면적"))
#     .sort("연월")
#     .collect()
# )


In [123]:
def agg_by_yearmonth(
    df: pl.LazyFrame,
    date_col: str,
    area_col: str,
    start: str = "20130101",
    end: str = "20221231",
) -> pl.DataFrame:
    new = ["0100"]  # 신축
    mod = ["0200", "0300", "0500", "0600"]  # 증축/개축/이전/대수선
    change_of_use = ["0700"]  # 용도변경

    if "건축_구분_코드" in df.columns:
        action = pl.col("건축_구분_코드").is_in(new + mod)
    elif "관리_주택대장_PK" in df.columns:  # 주택인허가대장은 행위구분이 없음
        action = True  # 주택인허가대장은 모두 신축(출처: 세움터 통계 쿼리)

    result = (
        df.filter(
            action
            & (pl.col(date_col).str.lengths() == 8)
            & (pl.col(date_col).str.slice(4, 2) >= "01")
            & (pl.col(date_col).str.slice(4, 2) <= "12")
            & (pl.col(date_col).str.slice(6, 2) >= "01")
            & (pl.col(date_col).str.slice(6, 2) <= "31")
            & (pl.col(date_col) >= start)
            & (pl.col(date_col) <= end)
        )
        .with_columns((pl.col(date_col).str.slice(0, 6).alias("연월")))
        .groupby("연월")
        .agg(pl.col(area_col).sum().alias("면적"))
        .sort("연월")
        .collect()
    )
    return result


In [124]:
df_permit_building = agg_by_yearmonth(df_result_building, "건축_허가_일", "층_면적(㎡)")
df_groundbreaking_building = agg_by_yearmonth(df_result_building, "실제_착공_일", "층_면적(㎡)")
df_completion_building = agg_by_yearmonth(df_result_building, "사용승인_일", "층_면적(㎡)")


In [59]:
df_permit_building.filter(pl.col("연월") >= "202201")


연월,면적
str,f64
"""202201""",8.6568e6
"""202202""",8.5927e6
"""202203""",1.0587e7
"""202204""",1.1136e7
"""202205""",1.3329e7
"""202206""",9.4472e6
"""202207""",9.7803e6
"""202208""",9.9126e6
"""202209""",8.9936e6


In [60]:
df_groundbreaking_building.filter(pl.col("연월") >= "202201")


연월,면적
str,f64
"""202201""",6.0766e6
"""202202""",5.1420e6
"""202203""",8.9975e6
"""202204""",8.4531e6
"""202205""",8.2808e6
"""202206""",7.3164e6
"""202207""",6.6631e6
"""202208""",6.7595e6
"""202209""",5.5494e6


In [61]:
df_completion_building.filter(pl.col("연월") >= "202201")


연월,면적
str,f64
"""202201""",6.4681e6
"""202202""",4.6415e6
"""202203""",6.0643e6
"""202204""",6.7721e6
"""202205""",6.6077e6
"""202206""",6.4318e6
"""202207""",5.5658e6
"""202208""",6.1266e6
"""202209""",5.7970e6


In [21]:
df_master_housing = pl.scan_parquet("data/주택인허가_기본개요_2022년_12월/*")
df_dong_housing = pl.scan_parquet("data/주택인허가_동별개요_2022년_12월/*")
df_floor_housing = pl.scan_parquet("data/주택인허가_층별개요_2022년_12월/*")
df_action_housing = pl.scan_parquet("data/주택인허가_행위개요_2022년_12월/*")


In [22]:
print(df_master_housing.columns)
print(df_dong_housing.columns)
print(df_floor_housing.columns)
print(df_action_housing.columns)


['관리_주택대장_PK', '대지_위치', '건물_명', '시군구_코드', '법정동_코드', '대지_구분_코드', '번', '지', '특수지_명', '블록', '로트', '용도_코드', '용도_코드_명', '구조_코드', '구조_코드_명', '주_건축물_수', '연면적(㎡)', '총_세대_수(세대)', '철거_멸실_구분_코드', '철거_멸실_구분_코드_명', '철거_시작_일', '철거_종료_일', '철거_멸실_일', '승인_일', '착공_예정_일', '착공_일', '사용_검사_예정_일', '사용_검사_일', '생성_일자', '__null_dask_index__']
['관리_동별_개요_PK', '관리_주택대장_PK', '대지_위치', '건물_명', '시군구_코드', '법정동_코드', '대지_구분_코드', '번', '지', '특수지_명', '블록', '로트', '주_부속_구분_코드', '주_부속_구분_코드_명', '동명', '주_용도_코드', '주_용도_코드_명', '세대_수_국민_임대(세대)', '세대_수_공공_임대_5(세대)', '세대_수_공공_임대_10(세대)', '세대_수_공공_임대_기타(세대)', '세대_수_공공_임대_계(세대)', '세대_수_공공_분양(세대)', '세대_수_사원_임대(세대)', '세대_수_근로_복지(세대)', '세대_수_민간_임대(세대)', '세대_수_민간_분양(세대)', '구조_코드', '구조_코드_명', '지붕_코드', '지붕_코드_명', '건축_면적(㎡)', '연면적(㎡)', '지하_면적(㎡)', '용적_률_산정_연면적(㎡)', '지하_층_수', '지상_층_수', '높이(m)', '승용_승강기_인용_수', '비상용_승강기_인용_수', '층고_FROM', '반자_높이(m)', '계단_유효_폭', '복도_너비', '외벽_두께', '인접_세대_벽_두께', '생성_일자', '__null_dask_index__']
['관리_층별_개요_PK', '관리_동별_개요_PK', '대지_위치', '건물_명', '시군구_코드', '법정동_코드', '대지

In [64]:
cols_master_housing = [
    "관리_주택대장_PK",
    # "연면적(㎡)",
    "철거_멸실_구분_코드",
    "철거_멸실_구분_코드_명",
    "철거_시작_일",
    "철거_종료_일",
    "철거_멸실_일",
    "승인_일",  # 대부분 값 있음
    "착공_예정_일",  # 절반 정도 있음
    "착공_일",  # 대부분 null
    "사용_검사_예정_일",
    "사용_검사_일",  # 절반 정도 있음
]
cols_dong_housing = [
    "관리_동별_개요_PK",
    "관리_주택대장_PK",
    "건물_명",
    "동명",
    "구조_코드",
    "구조_코드_명",
    # "연면적(㎡)",
]
cols_floor_housing = [
    "관리_층별_개요_PK",
    "관리_동별_개요_PK",
    "시군구_코드",
    "동명",
    "층_번호",
    "층_구분_코드",
    "층_구분_코드_명",
    "층_면적(㎡)",
    "용도_코드",
    "용도_코드_명",
]


In [65]:
df_master_housing.select(cols_master_housing).collect().describe()


describe,관리_주택대장_PK,철거_멸실_구분_코드,철거_멸실_구분_코드_명,철거_시작_일,철거_종료_일,철거_멸실_일,승인_일,착공_예정_일,착공_일,사용_검사_예정_일,사용_검사_일
str,str,str,str,str,str,str,str,str,str,str,str
"""count""","""233865""","""233865""","""233865""","""233865""","""233865""","""233865""","""233865""","""233865""","""233865""","""233865""","""233865"""
"""null_count""","""0""","""233172""","""233172""","""233255""","""233259""","""233810""","""439""","""172508""","""233846""","""159318""","""141087"""
"""mean""",null,null,null,null,null,null,null,null,null,null,null
"""std""",null,null,null,null,null,null,null,null,null,null,null
"""min""","""-100006082""","""1""","""멸실""","""20071001""","""20070320""","""20000101""","""0""","""02140224""","""20080410""","""00301""","""19780930"""
"""max""","""50130-9""","""4""","""철거""","""21031129""","""20200930""","""20190415""","""30000101""","""981201""","""20200618""","""97""","""20230920"""
"""median""",null,null,null,null,null,null,null,null,null,null,null
"""25%""",null,null,null,null,null,null,null,null,null,null,null
"""75%""",null,null,null,null,null,null,null,null,null,null,null


착공일이 19개 빼고 없다고?????????????????????????????

In [66]:
df_dong_housing.select(cols_dong_housing).collect().describe()


describe,관리_동별_개요_PK,관리_주택대장_PK,건물_명,동명,구조_코드,구조_코드_명
str,str,str,str,str,str,str
"""count""","""332074""","""332074""","""332074""","""332074""","""332074""","""332074"""
"""null_count""","""0""","""0""","""41282""","""1994""","""3520""","""3525"""
"""mean""",null,null,null,null,null,null
"""std""",null,null,null,null,null,null
"""min""","""-100004282""","""-100006082""",""" ******-******…","""# 401동""","""10""","""강파이프구조"""
"""max""","""50130-99""","""50130-9""","""？브르타운 공동주택 신축공…","""힘멜에듀센터어린이집""","""99""","""프리케스트콘크리트구조"""
"""median""",null,null,null,null,null,null
"""25%""",null,null,null,null,null,null
"""75%""",null,null,null,null,null,null


In [67]:
df_floor_housing.select(cols_floor_housing).collect().describe()


describe,관리_층별_개요_PK,관리_동별_개요_PK,시군구_코드,동명,층_번호,층_구분_코드,층_구분_코드_명,층_면적(㎡),용도_코드,용도_코드_명
str,str,str,str,str,f64,str,str,f64,str,str
"""count""","""2857418""","""2857418""","""2857418""","""2857418""",2.857418e6,"""2857418""","""2857418""",2.857418e6,"""2857418""","""2857418"""
"""null_count""","""0""","""0""","""1400""","""1559""",602.0,"""289""","""305""",0.0,"""1137""","""1929"""
"""mean""",null,null,null,null,11.28422,null,null,1021.854939,null,null
"""std""",null,null,null,null,31.990283,null,null,162599.672687,null,null
"""min""","""-100076108""","""-100019201""","""0700""","""# 401동""",-33.0,"""10""","""복수층(하층)""",-4821.2229,"""01000""","""가스배관시설"""
"""max""","""50130-99""","""50130-99""","""50130""","""힘멜에듀센터어린이집""",1215.0,"""39""","""지하""",2.18776697e8,"""n1300""","""휴게음식점"""
"""median""",null,null,null,null,9.0,null,null,426.4973,null,null
"""25%""",null,null,null,null,3.0,null,null,287.9918,null,null
"""75%""",null,null,null,null,15.0,null,null,516.1017,null,null


In [68]:
df_master_housing.select("연면적(㎡)").sum().collect().item()


153052569861.83148

In [69]:
df_dong_housing.select("연면적(㎡)").sum().collect().item()


4201841082.952662

In [70]:
df_floor_housing.select("층_면적(㎡)").sum().collect().item()


2919866695.465873

153,052,569,861 m²에서 2,919,866,695 m²로 2%만 남음.

In [71]:
first = (
    (pl.col("시군구_코드") == "36110")
    & (pl.col("법정동_코드") == "10600")
    & (pl.col("대지_구분_코드") == "0")
    & (pl.col("번") == "0939")
    & (pl.col("지") == "0000")
)


In [72]:
df_master_housing.filter(first).select(cols_master_housing).collect()


관리_주택대장_PK,철거_멸실_구분_코드,철거_멸실_구분_코드_명,철거_시작_일,철거_종료_일,철거_멸실_일,승인_일,착공_예정_일,착공_일,사용_검사_예정_일,사용_검사_일
str,str,str,str,str,str,str,str,str,str,str
"""36110-50000026…",null,null,null,null,null,"""20170914""",null,null,null,null
"""36110-50000039…",null,null,null,null,null,"""20181012""",null,null,null,"""20181120"""
"""36110-50000049…",null,null,null,null,null,"""20191129""",null,null,"""20191225""","""20191218"""
"""36110-50000061…",null,null,null,null,null,"""20201211""",null,null,"""20201230""","""20210105"""
"""36110-50000062…",null,null,null,null,null,"""20210105""",null,null,"""20210205""","""20210513"""


In [73]:
df_dong_housing.filter(first).select(cols_dong_housing).collect()


관리_동별_개요_PK,관리_주택대장_PK,건물_명,동명,구조_코드,구조_코드_명
str,str,str,str,str,str
"""36110-50000068…","""36110-50000039…",null,"""지하주차장""","""21""","""철근콘크리트구조"""


In [74]:
first_master_pk = (
    df_master_housing.filter(first).select("관리_주택대장_PK").collect().to_series().to_list()
)
first_master_pk


['36110-5000002622',
 '36110-5000003962',
 '36110-5000004982',
 '36110-5000006162',
 '36110-5000006242']

In [75]:
df_dong_housing.filter(pl.col("관리_주택대장_PK").is_in(first_master_pk)).select(
    cols_dong_housing
).collect()


관리_동별_개요_PK,관리_주택대장_PK,건물_명,동명,구조_코드,구조_코드_명
str,str,str,str,str,str
"""36110-50000068…","""36110-50000039…",null,"""지하주차장""","""21""","""철근콘크리트구조"""


똑같은데?

In [76]:
df_floor_housing.filter(first).select(cols_floor_housing).collect()


관리_층별_개요_PK,관리_동별_개요_PK,시군구_코드,동명,층_번호,층_구분_코드,층_구분_코드_명,층_면적(㎡),용도_코드,용도_코드_명
str,str,str,str,i64,str,str,f64,str,str


In [77]:
first_dong_pk = (
    df_dong_housing.filter(first).select("관리_동별_개요_PK").collect().to_series().to_list()
)
first_dong_pk


['36110-5000006821']

In [78]:
df_floor_housing.filter(pl.col("관리_동별_개요_PK").is_in(first_dong_pk)).select(
    cols_floor_housing
).collect()


관리_층별_개요_PK,관리_동별_개요_PK,시군구_코드,동명,층_번호,층_구분_코드,층_구분_코드_명,층_면적(㎡),용도_코드,용도_코드_명
str,str,str,str,i64,str,str,f64,str,str


똑같은데??? 없는데?????

In [79]:
df_dong_building.filter(first).select(cols_dong_building).collect()


관리_동별_개요_PK,관리_허가대장_PK,구조_코드,구조_코드_명
str,str,str,str


In [80]:
df_floor_building.filter(first).select(cols_floor_building).collect()


관리_층별_개요_PK,관리_동별_개요_PK,관리_허가대장_PK,시군구_코드,주_용도_코드,주_용도_코드_명,층_번호,층_면적(㎡),층_구분_코드,층_구분_코드_명,건축_구분_코드,건축_구분_코드_명
str,str,str,str,str,str,i64,f64,str,str,str,str


건축인허가대장에도 없긴 마찬가진데...

In [131]:
df_master_housing.filter(pl.col("관리_주택대장_PK").is_in(["36110-100004121"])).collect()


관리_주택대장_PK,대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,용도_코드,용도_코드_명,구조_코드,구조_코드_명,주_건축물_수,연면적(㎡),총_세대_수(세대),철거_멸실_구분_코드,철거_멸실_구분_코드_명,철거_시작_일,철거_종료_일,철거_멸실_일,승인_일,착공_예정_일,착공_일,사용_검사_예정_일,사용_검사_일,생성_일자,__null_dask_index__
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,f64,i64,str,str,str,str,str,str,str,str,str,str,str,i64
"""36110-10000412…","""세종특별자치시 나성동 블…","""첫마을 아파트""","""36110""","""10700""","""2""",null,null,""",송원리 일원(첫마을)""","""A1 BL""",null,"""02000""","""공동주택""",null,null,11,169706.83,895,null,null,null,null,null,"""20071227""","""20090301""",null,"""20111031""",null,"""20140821""",133578


In [130]:
df_dong_housing.filter(pl.col("관리_주택대장_PK").is_in(["36110-100004121"])).select(
    cols_dong_housing
).collect()


관리_동별_개요_PK,관리_주택대장_PK,건물_명,동명,구조_코드,구조_코드_명
str,str,str,str,str,str
"""36110-10000396…","""36110-10000412…","""첫마을 아파트""","""101동""","""21""","""철근콘크리트구조"""
"""36110-10000396…","""36110-10000412…","""첫마을 아파트""","""경로당""","""21""","""철근콘크리트구조"""
"""36110-10000396…","""36110-10000412…","""첫마을 아파트""","""주민공동시설-3""","""21""","""철근콘크리트구조"""
"""36110-10000396…","""36110-10000412…","""첫마을 아파트""","""110동""","""21""","""철근콘크리트구조"""
"""36110-10000396…","""36110-10000412…","""첫마을 아파트""","""주민공동시설-1""","""21""","""철근콘크리트구조"""
"""36110-10000396…","""36110-10000412…","""첫마을 아파트""","""108동""","""21""","""철근콘크리트구조"""
"""36110-10000396…","""36110-10000412…","""첫마을 아파트""","""경비실-1""","""21""","""철근콘크리트구조"""
"""36110-10000396…","""36110-10000412…","""첫마을 아파트""","""근린생활시설""","""21""","""철근콘크리트구조"""
"""36110-10000396…","""36110-10000412…","""첫마을 아파트""","""109동""","""21""","""철근콘크리트구조"""


주소가... 바뀌었구나

In [89]:
df_result_housing = (
    df_master_housing.select(cols_master_housing)
    .filter(  # 2022년까지 데이터만
        (pl.col("승인_일").str.lengths() == 8) & (pl.col("승인_일") <= "20221231")
    )
    .join(df_dong_housing.select(cols_dong_housing), "관리_주택대장_PK", suffix="_dong")
    .join(df_floor_housing.select(cols_floor_housing), "관리_동별_개요_PK", suffix="_floor")
)


In [90]:
df_result_housing.head().collect()


관리_주택대장_PK,철거_멸실_구분_코드,철거_멸실_구분_코드_명,철거_시작_일,철거_종료_일,철거_멸실_일,승인_일,착공_예정_일,착공_일,사용_검사_예정_일,사용_검사_일,관리_동별_개요_PK,건물_명,동명,구조_코드,구조_코드_명,관리_층별_개요_PK,시군구_코드,동명_floor,층_번호,층_구분_코드,층_구분_코드_명,층_면적(㎡),용도_코드,용도_코드_명
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,f64,str,str
"""45130-10000429…",null,null,null,null,null,"""20071031""","""20081023""",null,"""20110331""","""20110331""","""45130-10000368…","""수송2차 오투그란데""","""708동""","""21""","""철근콘크리트구조""","""45130-10000879…","""45130""","""708동""",16,"""20""","""지상""",328.9408,"""02001""","""아파트"""
"""45130-10000429…",null,null,null,null,null,"""20071031""","""20081023""",null,"""20110331""","""20110331""","""45130-10000368…","""수송2차 오투그란데""","""708동""","""21""","""철근콘크리트구조""","""45130-10000879…","""45130""","""708동""",17,"""20""","""지상""",264.54,"""02001""","""아파트"""
"""45130-10000429…",null,null,null,null,null,"""20071031""","""20081023""",null,"""20110331""","""20110331""","""45130-10000368…","""수송2차 오투그란데""","""708동""","""21""","""철근콘크리트구조""","""45130-10000880…","""45130""","""708동""",18,"""20""","""지상""",264.54,"""02001""","""아파트"""
"""45130-10000429…",null,null,null,null,null,"""20071031""","""20081023""",null,"""20110331""","""20110331""","""45130-10000368…","""수송2차 오투그란데""","""708동""","""21""","""철근콘크리트구조""","""45130-10000880…","""45130""","""708동""",19,"""20""","""지상""",264.54,"""02001""","""아파트"""
"""45130-10000429…",null,null,null,null,null,"""20071031""","""20081023""",null,"""20110331""","""20110331""","""45130-10000368…","""수송2차 오투그란데""","""708동""","""21""","""철근콘크리트구조""","""45130-10000880…","""45130""","""708동""",20,"""20""","""지상""",264.54,"""02001""","""아파트"""


In [125]:
df_permit_housing = agg_by_yearmonth(df_result_housing, "승인_일", "층_면적(㎡)")
df_groundbreaking_housing = agg_by_yearmonth(df_result_housing, "착공_일", "층_면적(㎡)")
df_completion_housing = agg_by_yearmonth(df_result_housing, "사용_검사_일", "층_면적(㎡)")


In [115]:
df_permit_housing.filter(pl.col("연월") >= "202201")


연월,면적
str,f64
"""202201""",6.1172e6
"""202202""",4.1392e6
"""202203""",5.6843e6
"""202204""",8.1688e6
"""202205""",1.5695e7
"""202206""",6.7948e6
"""202207""",6.7229e6
"""202208""",4.6275e7
"""202209""",5.5424e6


In [116]:
df_groundbreaking_housing.filter(pl.col("연월") >= "202201")  #### 1년 동안 착공이 없다고?


연월,면적
str,f64


In [117]:
df_completion_housing.filter(pl.col("연월") >= "202201")


연월,면적
str,f64
"""202201""",3.7825e6
"""202202""",1.8245e6
"""202203""",2.8075e6
"""202204""",4.5424e6
"""202205""",4.4403e6
"""202206""",3.7849e6
"""202207""",3.8513e6
"""202208""",5.1095e6
"""202209""",3.6833e7


In [126]:
df_permit = (
    pl.concat([df_permit_building, df_permit_housing])
    .groupby("연월")
    .agg(pl.col("면적").sum().alias("인허가"))
    .sort("연월")
)
df_groundbreaking = (
    pl.concat([df_groundbreaking_building, df_groundbreaking_housing])
    .groupby("연월")
    .agg(pl.col("면적").sum().alias("착공"))
    .sort("연월")
)
df_completion = (
    pl.concat([df_completion_building, df_completion_housing])
    .groupby("연월")
    .agg(pl.col("면적").sum().alias("준공"))
    .sort("연월")
)
df_result = (
    df_permit.join(df_groundbreaking, "연월", "outer")
    .join(df_completion, "연월", "outer")
    .sort("연월")
)


In [127]:
df_result


연월,인허가,착공,준공
str,f64,f64,f64
"""201301""",8.5849e6,4.3207e6,6.8075e6
"""201302""",7.8529e6,5.4231e6,8.2018e6
"""201303""",1.0497e7,6.3643e6,6.0214e6
"""201304""",1.0765e7,7.2707e6,8.5601e6
"""201305""",1.3459e7,7.0361e6,7.9618e6
"""201306""",1.3006e7,6.0840e6,1.2671e7
"""201307""",1.3777e7,6.9213e6,8.0384e6
"""201308""",1.2221e7,6.7267e6,8.6422e6
"""201309""",1.1558e7,5.8024e6,7.6727e6


In [132]:
df_result.rename(
    {"연월": "yearmonth", "인허가": "permit", "착공": "groundbreaking", "준공": "completion"}
).to_pandas().to_csv("output/permit.csv", encoding="utf-8-sig", index=False)


In [15]:
sejong = pl.col("시군구_코드").str.starts_with("36")


In [28]:
df_master_sejong = df_master.filter(sejong)
df_dong_sejong = df_dong.filter(sejong)
df_floor_sejong = df_floor.filter(sejong)


In [ ]:
df_master_sejong.head()


In [31]:
df_master_sejong_202212 = df_master_sejong.filter(
    pl.col("건축_허가_일").str.starts_with("202212")
)
df_master_sejong_202212 = df_master_sejong_202212.select(
    [
        "관리_허가대장_PK",
        "시군구_코드",
        "건축_구분_코드",
        "건축_구분_코드_명",
        "대지_면적(㎡)",
        "건축_면적(㎡)",
        "건폐_율(%)",
        "연면적(㎡)",
        "용적_률_산정_연면적(㎡)",
        "용적_률(%)",
    ]
)


In [49]:
df_dong_sejong_202212 = df_master_sejong_202212.join(df_dong_sejong, "관리_허가대장_PK")
df_dong_sejong_202212 = df_dong_sejong_202212.select(
    [
        "관리_허가대장_PK",
        "시군구_코드",
        "건축_구분_코드",
        "건축_구분_코드_명",
        "대지_면적(㎡)",
        "건축_면적(㎡)",
        "건폐_율(%)",
        "연면적(㎡)",
        "용적_률_산정_연면적(㎡)",
        "용적_률(%)",
        "관리_동별_개요_PK",
        "건물_명",
        "주_부속_구분_코드",
        "주_부속_구분_코드_명",
        "주_용도_코드",
        "주_용도_코드_명",
        "구조_코드",
        "구조_코드_명",
        "지붕_코드",
        "지붕_코드_명",
        "건축_면적(㎡)_right",
        "연면적(㎡)_right",
        "용적_률_산정_연면적(㎡)_right",
    ]
)
df_dong_sejong_202212 = df_dong_sejong_202212.rename(
    {
        "건축_면적(㎡)_right": "건축_면적(㎡)_동",
        "연면적(㎡)_right": "연면적(㎡)_동",
        "용적_률_산정_연면적(㎡)_right": "용적_률_산정_연면적(㎡)_동",
    }
)


In [50]:
df_dong_sejong_202212.head()


관리_허가대장_PK,시군구_코드,건축_구분_코드,건축_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),관리_동별_개요_PK,건물_명,주_부속_구분_코드,주_부속_구분_코드_명,주_용도_코드,주_용도_코드_명,구조_코드,구조_코드_명,지붕_코드,지붕_코드_명,건축_면적(㎡)_동,연면적(㎡)_동,용적_률_산정_연면적(㎡)_동
str,str,str,str,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,f64,f64,f64
"""36110-10000000…","""36110""",null,null,610.0,18.0,0.0,0.0,0.0,0.0,"""36110-10000000…",null,"""0""","""주건축물""","""28000""","""가설건축물""","""74""","""컨테이너조""",null,null,18.0,18.0,18.0
"""36110-10000000…","""36110""",null,null,622.0,18.0,0.0,18.0,0.0,0.0,"""36110-10000000…",null,"""0""","""주건축물""","""28000""","""가설건축물""","""74""","""컨테이너조""",null,null,18.0,18.0,18.0
"""36110-10000000…","""36110""",null,null,1653.0,60.0,0.0,60.0,0.0,0.0,"""36110-10000000…",null,"""0""","""주건축물""","""28000""","""가설건축물""","""33""","""강파이프구조""",null,null,60.0,60.0,60.0
"""36110-10000000…","""36110""",null,null,11407.4,99.0,0.0,99.0,0.0,0.0,"""36110-10000000…",null,"""0""","""주건축물""","""18000""","""창고시설""","""33""","""강파이프구조""",null,null,99.0,99.0,99.0
"""36110-10000000…","""36110""","""0200""","""증축""",3804.0,1634.15,42.96,1634.15,1634.15,42.96,"""36110-10000000…","""연동면 송용리 1028-1…","""0""","""주건축물""","""21000""","""동.식물관련시설""","""33""","""강파이프구조""","""90""","""기타지붕""",1382.15,1382.15,1382.15


In [77]:
df_dong_sejong_202212.groupby("건축_구분_코드").sum()


건축_구분_코드,관리_허가대장_PK,시군구_코드,건축_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),관리_동별_개요_PK,건물_명,주_부속_구분_코드,주_부속_구분_코드_명,주_용도_코드,주_용도_코드_명,구조_코드,구조_코드_명,지붕_코드,지붕_코드_명,건축_면적(㎡)_동,연면적(㎡)_동,용적_률_산정_연면적(㎡)_동
str,str,str,str,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,f64,f64,f64
null,null,null,null,2939652.4,16012.02,0.0,16654.58,0.0,0.0,null,null,null,null,null,null,null,null,null,null,4960.228,5302.228,5287.228
"""0100""",null,null,null,248611.0,62544.56,894.1005,102777.55,102356.5,1097.7287,null,null,null,null,null,null,null,null,null,null,33244.43,54907.86,54486.81
"""0700""",null,null,null,19728.0,12364.79,322.425,109317.571,66973.545,1550.3116,null,null,null,null,null,null,null,null,null,null,1433.14,135.98,10572.0
"""0200""",null,null,null,673008.5,59479.575,495.1454,174474.105,162738.045,939.7232,null,null,null,null,null,null,null,null,null,null,3438.237,6224.057,5755.657


In [51]:
df_floor_sejong.head()


관리_층별_개요_PK,관리_동별_개요_PK,관리_허가대장_PK,대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,구조_코드,구조_코드_명,주_용도_코드,주_용도_코드_명,층_번호,층_면적(㎡),층_구분_코드,층_구분_코드_명,건축_구분_코드,건축_구분_코드_명,생성_일자,__null_dask_index__
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,f64,str,str,str,str,str,i64
"""36110-50002046…","""36110-50002593…","""36110-50001466…","""세종특별자치시 전의면 읍…",null,"""36110""","""37021""","""0""","""0100""","""0005""",null,null,null,"""33""","""강파이프구조""","""28010""","""임시창고""",1,168.75,"""20""","""지상""","""0100""","""신축""","""20221108""",1035
"""36110-20000170…","""36110-20000116…","""36110-20000076…","""세종특별자치시 부강면 금…",null,"""36110""","""33028""","""0""","""0662""","""0000""",null,null,null,"""31""","""일반철골구조""","""17100""","""일반공장""",1,0.0,"""20""","""지상""","""2000""","""허가/신고사항변경""","""20230105""",1390
"""36110-50001386…","""36110-50001737…","""36110-50000926…","""세종특별자치시 전동면 송…",null,"""36110""","""38026""","""0""","""0311""","""0000""",null,null,null,"""32""","""경량철골구조""","""18101""","""일반창고""",1,135.0,"""20""","""지상""","""0100""","""신축""","""20230105""",1435
"""36110-50000171…","""36110-50000162…","""36110-50000101…","""세종특별자치시 나성동 블…",null,"""36110""","""10700""","""2""","""0000""","""0000""","""행정중심복합도시""","""2-4생활권""","""CB3-3""","""39""","""기타강구조""","""28009""","""임시사무실""",1,18.0,"""20""","""지상""","""0100""","""신축""","""20130820""",1957
"""36110-50000574…","""36110-50000612…","""36110-50000374…","""세종특별자치시 고운동 블…","""성진프라자""","""36110""","""11200""","""2""","""0000""","""0000""","""1-1생활권""","""C1""","""2""","""21""","""철근콘크리트구조""","""03020""","""체육도장""",7,64.68,"""20""","""지상""","""0700""","""용도변경""","""20230105""",2207


In [71]:
df_floor_sejong_202212 = df_dong_sejong_202212.join(df_floor_sejong, "관리_동별_개요_PK")
df_floor_sejong_202212 = df_floor_sejong_202212.select(
    [
        "관리_허가대장_PK",
        "시군구_코드",
        "건축_구분_코드",
        "건축_구분_코드_명",
        "대지_면적(㎡)",
        "건축_면적(㎡)",
        "건폐_율(%)",
        "연면적(㎡)",
        "용적_률_산정_연면적(㎡)",
        "용적_률(%)",
        "관리_동별_개요_PK",
        "건물_명",
        "주_부속_구분_코드",
        "주_부속_구분_코드_명",
        "주_용도_코드",
        "주_용도_코드_명",
        "구조_코드",
        "구조_코드_명",
        "지붕_코드",
        "지붕_코드_명",
        "건축_면적(㎡)_동",
        "연면적(㎡)_동",
        "용적_률_산정_연면적(㎡)_동",
        "관리_층별_개요_PK",
        "주_용도_코드_right",
        "주_용도_코드_명_right",
        "층_번호",
        "층_면적(㎡)",
        "층_구분_코드",
        "층_구분_코드_명",
        "건축_구분_코드_right",
        "건축_구분_코드_명_right",
    ]
)
df_floor_sejong_202212 = df_floor_sejong_202212.rename(
    {
        "주_용도_코드_right": "주_용도_코드_층",
        "주_용도_코드_명_right": "주_용도_코드_명_층",
        "건축_구분_코드_right": "건축_구분_코드_층",
        "건축_구분_코드_명_right": "건축_구분_코드_명_층",
    }
)


In [72]:
df_floor_sejong_202212.head()


관리_허가대장_PK,시군구_코드,건축_구분_코드,건축_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),관리_동별_개요_PK,건물_명,주_부속_구분_코드,주_부속_구분_코드_명,주_용도_코드,주_용도_코드_명,구조_코드,구조_코드_명,지붕_코드,지붕_코드_명,건축_면적(㎡)_동,연면적(㎡)_동,용적_률_산정_연면적(㎡)_동,관리_층별_개요_PK,주_용도_코드_층,주_용도_코드_명_층,층_번호,층_면적(㎡),층_구분_코드,층_구분_코드_명,건축_구분_코드_층,건축_구분_코드_명_층
str,str,str,str,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,str,str,str,i64,f64,str,str,str,str
"""36110-10000000…","""36110""",null,null,1200.0,18.0,0.0,18.0,0.0,0.0,"""36110-10000000…",null,"""0""","""주건축물""","""28000""","""가설건축물""","""74""","""컨테이너조""",null,null,18.0,18.0,18.0,"""36110-10000000…","""28004""","""공사용가설건축물""",1,18.0,"""20""","""지상""",null,null
"""36110-10000000…","""36110""","""0100""","""신축""",1002.0,120.0,11.98,120.0,120.0,11.98,"""36110-10000000…","""장군면 용암리 72-11 …","""0""","""주건축물""","""03000""","""제1종근린생활시설""","""32""","""경량철골구조""","""90""","""기타지붕""",120.0,120.0,120.0,"""36110-10000000…","""03001""","""소매점""",1,120.0,"""20""","""지상""","""0100""","""신축"""
"""36110-10000000…","""36110""",null,null,10464.7,12.0,0.0,12.0,0.0,0.0,"""36110-10000000…",null,"""0""","""주건축물""","""28999""","""기타가설건축물""","""74""","""컨테이너조""",null,null,12.0,12.0,12.0,"""36110-10000000…","""17301""","""물품 제조공장""",1,12.0,"""20""","""지상""",null,null
"""36110-10000000…","""36110""",null,null,3453.0,395.9,0.0,395.9,0.0,0.0,"""36110-10000000…",null,"""0""","""주건축물""","""28000""","""가설건축물""","""33""","""강파이프구조""",null,null,207.9,207.9,207.9,"""36110-10000000…","""28010""","""임시창고""",1,207.9,"""20""","""지상""",null,null
"""36110-10000000…","""36110""",null,null,2810.8,161.23,0.0,161.23,0.0,0.0,"""36110-10000000…",null,"""0""","""주건축물""","""28000""","""가설건축물""","""32""","""경량철골구조""",null,null,161.238,161.238,161.238,"""36110-10000000…","""28008""","""임시자동차차고""",1,161.238,"""20""","""지상""",null,null


In [75]:
df_floor_sejong_202212.sum()


관리_허가대장_PK,시군구_코드,건축_구분_코드,건축_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),관리_동별_개요_PK,건물_명,주_부속_구분_코드,주_부속_구분_코드_명,주_용도_코드,주_용도_코드_명,구조_코드,구조_코드_명,지붕_코드,지붕_코드_명,건축_면적(㎡)_동,연면적(㎡)_동,용적_률_산정_연면적(㎡)_동,관리_층별_개요_PK,주_용도_코드_층,주_용도_코드_명_층,층_번호,층_면적(㎡),층_구분_코드,층_구분_코드_명,건축_구분_코드_층,건축_구분_코드_명_층
str,str,str,str,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,str,str,str,i64,f64,str,str,str,str
null,null,null,null,5540752.3,431326.465,3003.2183,1.2589e6,1.1362e6,7311.6733,null,null,null,null,null,null,null,null,null,null,163298.935,287355.875,303364.465,null,null,null,441,71483.148,null,null,null,null


In [76]:
df_floor_sejong_202212.groupby("건축_구분_코드").sum()


건축_구분_코드,관리_허가대장_PK,시군구_코드,건축_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),관리_동별_개요_PK,건물_명,주_부속_구분_코드,주_부속_구분_코드_명,주_용도_코드,주_용도_코드_명,구조_코드,구조_코드_명,지붕_코드,지붕_코드_명,건축_면적(㎡)_동,연면적(㎡)_동,용적_률_산정_연면적(㎡)_동,관리_층별_개요_PK,주_용도_코드_층,주_용도_코드_명_층,층_번호,층_면적(㎡),층_구분_코드,층_구분_코드_명,건축_구분_코드_층,건축_구분_코드_명_층
str,str,str,str,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,str,str,str,i64,f64,str,str,str,str
"""0700""",null,null,null,22036.0,13894.64,442.0437,124928.761,77748.505,2180.1916,null,null,null,null,null,null,null,null,null,null,2781.52,135.98,21008.02,null,null,null,29,4706.203,null,null,null,null
"""0200""",null,null,null,1893440.5,224541.585,1181.6845,815083.165,758518.645,3062.653,null,null,null,null,null,null,null,null,null,null,10818.247,30470.257,27619.057,null,null,null,160,6263.447,null,null,null,null
"""0100""",null,null,null,659648.0,176678.97,1379.4901,301912.0,299914.75,2068.8287,null,null,null,null,null,null,null,null,null,null,144593.69,251257.16,249259.91,null,null,null,90,55208.27,null,null,null,null
null,null,null,null,2965627.8,16211.27,0.0,16943.83,0.0,0.0,null,null,null,null,null,null,null,null,null,null,5105.478,5492.478,5477.478,null,null,null,162,5305.228,null,null,null,null


In [85]:
df_floor_sejong_202212.groupby(["건축_구분_코드_명", "구조_코드_명"]).sum().sort(
    ["건축_구분_코드_명", "구조_코드_명"]
)


건축_구분_코드_명,구조_코드_명,관리_허가대장_PK,시군구_코드,건축_구분_코드,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),관리_동별_개요_PK,건물_명,주_부속_구분_코드,주_부속_구분_코드_명,주_용도_코드,주_용도_코드_명,구조_코드,지붕_코드,지붕_코드_명,건축_면적(㎡)_동,연면적(㎡)_동,용적_률_산정_연면적(㎡)_동,관리_층별_개요_PK,주_용도_코드_층,주_용도_코드_명_층,층_번호,층_면적(㎡),층_구분_코드,층_구분_코드_명,건축_구분_코드_층,건축_구분_코드_명_층
str,str,str,str,str,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,f64,f64,f64,str,str,str,i64,f64,str,str,str,str
null,"""강파이프구조""",null,null,null,39733.2,2317.12,0.0,2317.12,0.0,0.0,null,null,null,null,null,null,null,null,null,1114.06,1114.06,1114.06,null,null,null,11,1022.81,null,null,null,null
null,"""경량철골구조""",null,null,null,3773.8,192.94,0.0,192.94,0.0,0.0,null,null,null,null,null,null,null,null,null,192.948,192.948,192.948,null,null,null,2,192.948,null,null,null,null
null,"""기타강구조""",null,null,null,71511.0,45.0,0.0,45.0,0.0,0.0,null,null,null,null,null,null,null,null,null,30.0,30.0,30.0,null,null,null,3,30.0,null,null,null,null
null,"""막구조""",null,null,null,8451.0,400.0,0.0,400.0,0.0,0.0,null,null,null,null,null,null,null,null,null,400.0,400.0,400.0,null,null,null,1,400.0,null,null,null,null
null,"""조립식판넬조""",null,null,null,296907.4,332.17,0.0,332.17,0.0,0.0,null,null,null,null,null,null,null,null,null,298.17,298.17,298.17,null,null,null,5,298.17,null,null,null,null
null,"""컨테이너조""",null,null,null,2545251.4,12924.04,0.0,13656.6,0.0,0.0,null,null,null,null,null,null,null,null,null,3070.3,3457.3,3442.3,null,null,null,140,3361.3,null,null,null,null
"""신축""","""경량철골구조""",null,null,null,33596.0,4383.63,517.9981,4342.28,4270.28,502.0187,null,null,null,null,null,null,null,null,null,2667.09,2645.84,2573.84,null,null,null,31,2517.62,null,null,null,null
"""신축""","""일반철골구조""",null,null,null,104613.0,28568.23,162.76,47157.89,47157.89,204.65,null,null,null,null,null,null,null,null,null,1832.99,1832.99,1832.99,null,null,null,7,1832.99,null,null,null,null
"""신축""","""철골콘크리트구조""",null,null,null,479380.0,134502.2,278.85,226908.5,226908.5,470.5,null,null,null,null,null,null,null,null,null,131274.2,223680.5,223680.5,null,null,null,22,45035.63,null,null,null,null


In [87]:
df_202212 = (
    df_master.filter(pl.col("건축_허가_일").str.starts_with("202212"))
    .join(df_dong, "관리_허가대장_PK", suffix="_동")
    .join(df_floor, "관리_동별_개요_PK", suffix="_층")
)


In [104]:
df_202212.select(
    [
        "관리_허가대장_PK",
        "관리_동별_개요_PK",
        "관리_층별_개요_PK",
        "주_용도_코드",
        "주_용도_코드_명",
        "주_용도_코드_층",
        "주_용도_코드_명_층",
        "층_면적(㎡)",
    ]
).sort("관리_허가대장_PK").head(20)


관리_허가대장_PK,관리_동별_개요_PK,관리_층별_개요_PK,주_용도_코드,주_용도_코드_명,주_용도_코드_층,주_용도_코드_명_층,층_면적(㎡)
str,str,str,str,str,str,str,f64
"""11000-10000000…","""11000-10000000…","""11000-10000000…","""02000""","""공동주택""","""02005""","""부대시설""",239.9274
"""11000-10000000…","""11000-10000000…","""11000-10000000…","""02000""","""공동주택""","""02001""","""아파트""",678.4546
"""11000-10000000…","""11000-10000000…","""11000-10000000…","""02000""","""공동주택""","""02001""","""아파트""",703.3728
"""11000-10000000…","""11000-10000000…","""11000-10000000…","""02000""","""공동주택""","""02001""","""아파트""",64.71
"""11000-10000000…","""11000-10000000…","""11000-10000000…","""02000""","""공동주택""","""02005""","""부대시설""",229.0583
"""11000-10000000…","""11000-10000000…","""11000-10000000…","""02000""","""공동주택""","""02001""","""아파트""",1412.3156
"""11000-10000000…","""11000-10000000…","""11000-10000000…","""02000""","""공동주택""","""02001""","""아파트""",678.4546
"""11000-10000000…","""11000-10000000…","""11000-10000000…","""02000""","""공동주택""","""02001""","""아파트""",243.4553
"""11000-10000000…","""11000-10000000…","""11000-10000000…","""02000""","""공동주택""","""02001""","""아파트""",633.6946


In [107]:
df_202212.select(["주_용도_코드_층", "주_용도_코드_명_층", "층_면적(㎡)"]).with_columns(
    pl.col("주_용도_코드_층").str.slice(0, 2).alias("층_용도_코드_대분류")
).groupby(["층_용도_코드_대분류"]).sum()


층_용도_코드_대분류,주_용도_코드_층,주_용도_코드_명_층,층_면적(㎡)
str,str,str,f64
"""28""",null,null,781774.5337
"""23""",null,null,59647.3275
"""10""",null,null,328070.8959
"""29""",null,null,1980.29
"""04""",null,null,849100.44677
"""07""",null,null,108453.6994
"""02""",null,null,722936.0854
"""31""",null,null,5631.27
"""12""",null,null,11208.94
